In [1]:
from rdkit import Chem

In [ ]:
file_path = '/content/drive/MyDrive/grad_school/thesis/mass_spec/data/MoNA-export-GC-MS_Spectra.sdf'
suppl = Chem.SDMolSupplier(file_path)

In [ ]:
def format_spectrum(chem):
    spectrum = chem.GetProp("MASS SPECTRAL PEAKS")
    spectrum = spectrum.split(' ')
    spectrum = [item.split('\n') for item in spectrum]
    spectrum = [i for item in spectrum for i in item]

    # the weights are currently at even indices in the list and their
    # frequencies are at odd. Dividing into two lists
    weights = []
    frequencies = []
    for i, num in enumerate(spectrum):
      if i % 2 == 0:
        num = round(float(num))
        weights.append(num)
      else:
        frequencies.append(float(num))

    # some of the spectra have a base peak of 1 instead of 100. Scaling those spectra here
    base_peak = max(frequencies)
    if base_peak != 100:
      frequencies = [freq/base_peak * 100 for freq in frequencies]


    # creating list to hold frequency of each weight - with 0s for weights with no data
    max_length = max(weights)
    weight_frequencies = [0] * max_length

    # filling in with mass spec data for the specific chemical
    for i in range(len(weight_frequencies)):
      if i+1 in weights:
        index = weights.index(i+1)
        weight_frequencies[i] = frequencies[index]

    return weight_frequencies, max_length

In [ ]:
# Trying to split this up into a few steps
# Getting smiles and instrument type counts for all chemicals in the dataset

new_chems_dict = {}
smiles_dict = {}
# some spectra have incorrect smiles. Keeping track of which chemicals have more than one smiles listed
smiles_counts = {}
all_chem_names = []

for chem in suppl:
  if chem is not None:
    # getting mass spec data for each chemical
    # only counting spectra with instrument type listed
    try:
      instrument_type = chem.GetProp('INSTRUMENT TYPE')
      if 'Pegasus' in instrument_type:
        instrument_type = 'GC-EI-TOF'
    except:
      continue

    try:
      unformatted_contributor =  chem.GetProp('CONTRIBUTOR')
      contributor = unformatted_contributor.split('{')[0]+unformatted_contributor.split('{')[1]
    except:
      contributor =  'Not Listed'

    name = chem.GetProp("NAME").title()
    all_chem_names.append(name)
    # add chemical's SMILES to smiles_dict
    comment = chem.GetProp('COMMENT').split('\n')
    for line in comment:
      if line.split('=')[0] == 'SMILES':
        smiles = line.split('SMILES=')[1]
        if smiles not in smiles_dict.keys():
          # there might already be a different smiles recorded for a chemical name. Fixing that below.
          if name not in smiles_dict.values():
            smiles_dict[smiles]=name
            break

    if name not in smiles_counts.keys():
      smiles_counts[name]={}
    if smiles not in smiles_counts[name].keys():
      smiles_counts[name][smiles] = 1
    else:
      smiles_counts[name][smiles] += 1

    # For chemicals that have the same smiles but different names, change so they have the same name
    if name not in smiles_dict.values():
      name = smiles_dict[smiles].title()

    if name not in new_chems_dict.keys():
      new_chems_dict[name] = {}
      new_chems_dict[name]['Instrument Type'] = {}
      # new_chems_dict[name]['Contributors'] = {}

      new_chems_dict[name]['Instrument Type List'] = []
      new_chems_dict[name]['Contributor List'] = []
      new_chems_dict[name]['count'] = 1
      new_chems_dict[name]['spectra'] = []
      new_chems_dict[name]['length'] = 0

    else:
      new_chems_dict[name]['count'] += 1

    spectrum, max_length = format_spectrum(chem)
    new_chems_dict[name]['spectra'].append(spectrum)
    new_chems_dict[name]['Instrument Type List'].append(instrument_type)
    new_chems_dict[name]['Contributor List'].append(contributor)
    if max_length > new_chems_dict[name]['length']:
      new_chems_dict[name]['length'] = max_length

    if instrument_type not in new_chems_dict[name]['Instrument Type'].keys():
      new_chems_dict[name]['Instrument Type'][instrument_type] = 1
    else:
      new_chems_dict[name]['Instrument Type'][instrument_type] += 1

    # if contributor not in new_chems_dict[name]['Contributors'].keys():
    #   new_chems_dict[name]['Contributors'][contributor] = 1
    # else:
    #   new_chems_dict[name]['Contributors'][contributor] += 1